![](https://logos-download.com/wp-content/uploads/2016/03/Netflix_logo.png)

 # 📽‍️ Netflix EDA for beginners 📺

> Netflix is a streaming service that offers a wide variety of award-winning TV shows, movies, anime, documentaries, and more on thousands of internet-connected devices. You can watch as much as you want, whenever you want without a single commercial – all for one low monthly price.

----------------------------------------

# **1. Introduction**

**Libraries** 📚⬇

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import spacy
from wordcloud import WordCloud

# Read the .csv files 📁 and create a copy of the dataframe 📃 

> 📌 Note:
> - The [**file.csv**](https://it.wikipedia.org/wiki/Comma-separated_values) contains all the information we need to analyze a bit of everything we know about Netflix and the products within it. It contains 29 unique columns and 15480 rows
> - I create a **copy of the [dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)** so that I can use and modify it without touching the original file which could always serve as a backup  

In [ ]:
data = pd.read_csv('../input/netflix-rotten-tomatoes-metacritic-imdb/netflix-rotten-tomatoes-metacritic-imdb.csv')
df = data.copy()
df.head()

> 📝 Let's take a look at our dataframe so we can open a few things:
> - let's start by observing what [**types of data**](https://realpython.com/python-data-types/) we have inside it with the **.info ()** function;

In [ ]:
df.info()

> - Let's continue our analysis by checking if and how many [**missing values**](https://en.wikipedia.org/wiki/Missing_data) there are; 

In [ ]:
df.isnull().sum()

> - As we can see there are many missing values in the various features so I decide to keep the most important columns in my opinion, even if they have different missing values that we will then remove, and to **drop** other **columns** that I consider superfluous for the analysis also for the large number of missing values they have;

In [ ]:
df = df.drop(columns = [ 'Metacritic Score', 'Boxoffice', 'Production House', 'Netflix Link', 'IMDb Link',
        'Poster', 'TMDb Trailer', 'Trailer Site'], axis = 1)

> - Now since we have 2 features (**'Release Date'** and **'Netflix Release Date'**) indicating a date as object datatype, I 'transform' them into **datetime features** so that we can use them as real dates;

In [ ]:
df['Release Date']= pd.to_datetime(df['Release Date'])
df['Netflix Release Date']= pd.to_datetime(df['Netflix Release Date'])

> - After this transformation of datatype for these 2 features, I create **2 new features** (one for the released year and one for the released year on Netflix) happy only for the year so as to be able to subsequently emerge an analysis on the productivity of the TV Series and Movie based on the various years;

In [ ]:
df['Released_Year'] = pd.DatetimeIndex(df['Release Date']).year
df['Released_Year_Net'] = pd.DatetimeIndex(df['Netflix Release Date']).year

> - Finally, let's take a **look at our dataframe** after the various changes we've made to it.

In [ ]:
df.head(1)

A **little description** of the features:
> * **Title**: Tells us the name of the movie / series
> * **Genre**: Describes the genre of the movie / series
> * **Tag**: is a list of tags on the movie / series
> * **Language**: Indicates in which language the film / series is available
> * **Series or Movie**: Indicates whether this product is a TV series or a movie
> * **Hidden Gem Score**: This is a score that suggests if this movie / series is a hidden gem (i.e. something that is not mainstream but is a great product and may deserve more attention)
> * **Country Availability**: Please indicate in which country this film / series is available
> * **Runtime**: Indicates the duration of the movie or an episode of the series
> * **Director**: Tells us how is the director of this product
> * **Writer**: Tells us how is the writer of this product,
> * **Actors**: Tells us how are the main actors of this product,
> * **View Rating**: Indicates the rating of the product,
> * **IMDb Score**: This is the score given to the product by IMDb,
> * **Rotten Tomatoes Score**: This is the score given to the product by Rotten Tomatoes,
> * **Awards Received**: It tells us how much awards this film/serie has received ,
> * **Awards Nominated For**: It tells us for which awards this film/movie has been nominated,
> * **Release Date**: It indicates when this product has been released,
> * **Netflix Release Date**: It indicates when this product has been released on Netflix,
> * **Summary**: It tells us a little summary of the plot of the product,
> * **IMDb Votes**: It indicates the scores given by IMDb,
> * **Image**: An Image
> * **Released_Year**: It indicates the year the product was released
> * **Released_Year_Net**: It indicates the year the product was released on Netflix

------------------------

# **2. Easy Data Analysis 🔬🧐**

# 2.1 Movie 📽‍️ vs Tv Series 📺

> 📌 Note:
> The **2 large macro categories** of products that we can find in the Netflix catalog are Movies and TV Series.
> - A **[movie](https://simple.wikipedia.org/wiki/Movie#:~:text=A%20movie%2C%20or%20film%2C%20is%20a%20type%20of,of%20entertainment%20or%20a%20way%20to%20have%20fun.)**, or film, is a type of visual communication which uses moving pictures and sound to tell stories or teach people something. Most people watch (view) movies as a type of entertainment or a way to have fun;
> - A television show is also called a television program (British English: programme), especially if it lacks a narrative structure. A **[television series](https://en.wikipedia.org/wiki/Television_show)** is usually released in episodes that follow a narrative, and are usually divided into seasons (US and Canada) or series (UK) – yearly or semiannual sets of new episodes.

So, knowing the presence of these 2 macro categories, now **let's figure out which of the 2 is more present** in the Netflix product catalog.

In [ ]:
colors = ['black',] * 2
colors[0] = 'crimson'

count = df['Series or Movie'].value_counts()

fig = go.Figure(data=[go.Bar(
    x = df["Series or Movie"],
    y = count,
    text = count,
    textposition='auto',
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'More Movie 📽‍️ or Tv Series 📺?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  barmode='group', xaxis_tickangle=-45,
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Category',
                  titlefont_size=14))

> As we can see from this [bar graph](https://plotly.com/python/bar-charts/) there is a big difference between the 2 categories as there are almost 12 thousand TV series against only 3.8 films. 

**Division in 2 dataframes 📄 📄**

> - After having ascertained this great difference between the 2 categories, we can **divide our dataframe into 2** so as to be able to work and analyze the TV Series and Movies in a distinct way.

> 

In [ ]:
df_movie = df[df['Series or Movie']=='Movie']
df_movie.head(1)

In [ ]:
df_series = df[df["Series or Movie"] == "Series"]
df_series.head(1)

--------------

# 2.2 Genres 😍😱😂

> 📌 Note:
> As we all know the world of cinema and TV series is very wide and boundless, this breadth in addition to the myriad of topics to be dealt with is also given by **the multitude of [film genres](https://www.masterclass.com/articles/how-to-identify-film-genres#:~:text=Some%20basic%20movie%20genres%20include%3A%201%20Action%3A%20Movies,pursuing%20an%20outlaw%20in%20the%20Wild%20West.%20)** that have been created over time by following a little also what was happening / happened in the literary field some time before. But **which of these is the most frequent and consequently the most requested by the spectators?** Will it be horror or action movies? Comedy or sentimental films? 

Let's find out right away with another bar chart.

**In Tv Series 📺...**

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Genre" column**.

In [ ]:
df_series_gen = df_series.dropna(subset=['Genre'])

> - Secondly **I create a [list](https://www.w3schools.com/python/python_lists.asp#:~:text=%20Python%20Lists%20%201%20List.%20Lists%20are,Duplicates.%20%205%20List%20Length.%20%20More%20)** that contains all the genres in the column and their quantity. So **I create a new dataframe** with this datas.

In [ ]:
colors_10 = ['DarkRed', 'FireBrick','Red', 'Crimson', 'IndianRed', 'slategray', 'gray', 'dimgrey', 'DarkSlateGrey', 'black']
series_gen_list = df_series_gen.Genre.str.split(',') #split the list into names
s_gen_list = {} #create an empty list
for genres in series_gen_list: # for any names in series_gen_list
    for genre in genres: # for any genre in genres
        if (genre in s_gen_list): #if this genre is already present in the s_gen_list
            s_gen_list[genre]+=1 # increase his value
        else:  # else
            s_gen_list[genre]=1 # Create his index in the list
s_gen_df = pd.DataFrame(s_gen_list.values(),index = s_gen_list.keys(),
                        columns = {'Counts of Genres in Tv Series'}) #Create a s_gen_df
s_gen_df.sort_values(by = 'Counts of Genres in Tv Series',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_s_gen = s_gen_df[0:10] 

> - Thirdly, **I draw a bar chart** to better understand the distribution of the quantities of the various genres contained in the Netflix catalog.

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_s_gen.index,
    y = top_10_s_gen['Counts of Genres in Tv Series'],
    text = top_10_s_gen['Counts of Genres in Tv Series'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most popular genres in Tv Series?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Genres',
                  titlefont_size=14))

**... We can see that the most present genres, and I think also most requested by viewers, are Drama 😟, Animation 🧚 and Comedies 😂.**

----------------------------

**Meanwhile in the Movie 📽‍️ world ...**

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Genre" column**.

In [ ]:
df_movie_gen = df_movie.dropna(subset=['Genre'])

> - Secondly **I create a list** that contains all the genres in the column and their quantity. So **I create a new dataframe** with this datas.

In [ ]:
movie_gen_list = df_movie_gen.Genre.str.split(', ') #split the list into names
m_gen_list = {} #create an empty list
for genres in movie_gen_list: # for any genres in movie_gen_list
    for genre in genres: # for any genre in genres
        if (genre in m_gen_list): #if this name is already present in the m_gen_list
            m_gen_list[genre]+=1 # increase his value
        else:  # else
            m_gen_list[genre]=1 # Create his index in the list
m_gen_df = pd.DataFrame(m_gen_list.values(),index = m_gen_list.keys(),
                        columns = {'Counts of Genres in Movies'}) #Create a m_gen_df
m_gen_df.sort_values(by = 'Counts of Genres in Movies',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_m_gen = m_gen_df[0:10] 

> - Thirdly, **I draw a bar chart** to better understand the distribution of the quantities of the various genres contained in the Netflix catalog.

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_gen.index,
    y = top_10_m_gen['Counts of Genres in Movies'],
    text = top_10_m_gen['Counts of Genres in Movies'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most popular genres in Movies?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Genres',
                  titlefont_size=14))

**... We can see that like with the Tv Series the Genre most rappresented is Drama 😟 followed by Comedy 😂 (that was third in the previous chart) and Thriller🕵🧟.**

-------------

# 2.3 Tags 📝📎 

> 📌 Note: As we are used to seeing on social networks, there are also **tags** in other fields. They can be considered as keywords that help to identify a certain thing in a simple and concise way. In our case they recall a little film genres but with slight nuances as maybe a film, or a TV series, can be considered as a set of different genres or in the tags the nationality of production or other small things that help the viewers in their search. 

So let's find out, even in this case, which are the most frequent and if there are close links with the quantity of products of the various genres that we have analyzed earlier.

**In the Tv Series 📺...**

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Tags" column**.

In [ ]:
df_series_tag = df_series.dropna(subset=['Tags'])

> - Secondly, I draw a **Wordcloud** with the most named tags in the list

In [ ]:
nlp = spacy.load('en')
series_tags = df_series_tag['Tags']
for tag in series_tags:
    doc = nlp(tag)
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]
rest_string = ' '
for i in words:
   rest_string = rest_string+i+' '
    
wordcloud = WordCloud(width = 1400, height = 1400, 
                background_color ='lavenderblush',  
                min_font_size = 5).generate(rest_string) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

**While in the Movie 📽‍️ world ..**.

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Tags" column**.

In [ ]:
df_movie_tag = df_movie.dropna(subset=['Tags'])

> - Secondly, I draw a **Wordcloud** with the most named tags in the list

In [ ]:
nlp = spacy.load('en')
movie_tags = df_movie_tag['Tags']
for tag in movie_tags:
    doc = nlp(tag)
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]
rest_string = ' '
for i in words:
   rest_string = rest_string+i+' '
    
wordcloud = WordCloud(width = 1400, height = 1400, 
                background_color ='lavenderblush',  
                min_font_size = 5).generate(rest_string) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

---------------

# 2.4 Languages 🗣🎤

> 📌 Note: There are hundreds of states in the world that rightfully have their own traditions, their own culture and their own languages. This is why there are, According to Ethnologue, **currently [7,117 languages ](https://www.theintrepidguide.com/how-many-languages-are-there-in-the-world/)spoken around the world.**. But what does this have to do with our argument? It really hits the spot because this being the situation, every film shot in a different country will be shot in the original language of that country. Fortunately, nowadays there are voice actors who translate them into various languages so as to give as many people as possible the opportunity to enjoy and watch the various films. 

But unfortunately not all films are so famous that they are translated for everyone, so let's go to the discovery of the languages most present in the cinema scene

**In Tv Series 📺...**

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Languages" column**.

In [ ]:
df_series_lan = df_series.dropna(subset=['Languages'])

> - Secondly plot a treemap that contains squares of different shapes that illustrate the languages spoken in the Tv Series

In [ ]:
ax= px.treemap(df_series_lan,path=["Languages"])
ax.show()

**While in the Movie 📽‍️ universe...**

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Languages" column**.

In [ ]:
df_movie_lan = df_movie.dropna(subset=['Languages'])

> - Secondly **I create a list** that contains the **top 10 languages** that are spoken in the Tv Series.



In [ ]:
ax= px.treemap(df_movie_lan,path=["Languages"])
ax.show()

-------------------------

# 2.5 Best Hidden Gems 🤠💎

> 📌 Note: Every time you finish watching a very fascinating TV series or movie, you are immediately looking for another similar or equally beautiful one to see. Keep it up for a while, until you run out of all the famous products and then what? How you do it? it is always difficult to understand if a film with few reviews really deserves but luckily for us in this dataset there is a feature called "**Hidden Gem Score**" which indicates a score that indicates **the films with fewer reviews but with very high ratings**.

So let's go immediately to find out which are these hidden gems in our 2 big categories so that if you need a help when you don't know what to choose you can come back from here 😉

> Code: 
> - First of all I create a new series dataframe **dropping the missing values contained in the "Hidden Gem Score" column**.
> - Secondly, I have sorted it by discending so the first values have the most high scores
> - Thirdly, I have create a list with the first 10 elements
> - And Finally I have plot the list

**For the Tv Series 📺:**

In [ ]:
df_series_hdg = df_series.dropna(subset=['Hidden Gem Score'])
df_series_hdg = df_series_hdg.sort_values(by = 'Hidden Gem Score', ascending = False)
top_s_10_list = df_series_hdg[:10]
top_s_10_list.reset_index(drop=True, inplace=True)
print('The 10 Best Hidden Gems on Tv Series are:\n',top_s_10_list['Title'])

**For the movie 📽‍️:**

In [ ]:
df_movie_hdg = df_movie.dropna(subset=['Hidden Gem Score'])
df_movie_hdg = df_movie_hdg.sort_values(by = 'Hidden Gem Score', ascending = False)
top_m_10_list = df_movie_hdg[:10]
top_m_10_list.reset_index(drop=True, inplace=True)
print('The 10 Best Hidden Gems in Movies are:\n',top_m_10_list['Title'])

--------------------

# 2.6 Countries 🌍 🗺

> 📌 Note: As we said with regard to languages, it also applies to countries as there are **[195](https://www.worldometers.info/geography/how-many-countries-are-there-in-the-world/) countries around the world**. But how many of them have the opportunity to produce products such as films or TV series? Which are the most productive? And which countries have the most movies or TV series available on Netflix?

Let's go to discover it

**In the Series Universe 📺...**

> Code:

> - First of all I create a new series dataframe **dropping the missing values contained in the "Country Availability" column**.

In [ ]:
df_series_ctr = df_series.dropna(subset=['Country Availability'])

> - Secondly **I create a list** that contains all the genres in the column and their quantity. So **I create a new dataframe** with this datas.

In [ ]:
series_ctr_list = df_series_ctr['Country Availability'].str.split(', ') #split the list into names
s_ctr_list = {} #create an empty list
for countries in series_ctr_list: # for any countries in series_ctr_list
    for country in countries: # for any country in countries
        if (country in s_ctr_list): #if this country is already present in the s_ctr_list
            s_ctr_list[country]+=1 # increase his value
        else:  # else
            s_ctr_list[country]=1 # Create his index in the list
s_ctr_df = pd.DataFrame(s_ctr_list.values(),index = s_ctr_list.keys(),
                        columns = {'Counts of Country Availability'}) #Create a s_ctr_df
s_ctr_df.sort_values(by = 'Counts of Country Availability',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_s_ctr = s_ctr_df[0:10] 

> - Thirdly, **I draw a bar chart** to better understand which countries have a larger availability about the Netflix Series.

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_s_ctr.index,
    y = top_10_s_ctr['Counts of Country Availability'],
    text = top_10_s_ctr['Counts of Country Availability'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which countries have a larger Series Availability?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Countries',
                  titlefont_size=14))

**And in the Movie world 📽‍️**

> Code:

> - First of all I create a new series dataframe **dropping the missing values contained in the "Country Availability" column**.

In [ ]:
df_movie_ctr = df_movie.dropna(subset=['Country Availability'])

> - Secondly **I create a list** that contains all the genres in the column and their quantity. So **I create a new dataframe** with this datas.

In [ ]:
movie_ctr_list = df_movie_ctr['Country Availability'].str.split(', ') #split the list into names
m_ctr_list = {} #create an empty list
for countries in movie_ctr_list: # for any countries in movie_ctr_list
    for country in countries: # for any country in countries
        if (country in m_ctr_list): #if this country is already present in the m_ctr_list
            m_ctr_list[country]+=1 # increase his value
        else:  # else
            m_ctr_list[country]=1 # Create his index in the list
m_ctr_df = pd.DataFrame(m_ctr_list.values(),index = m_ctr_list.keys(),
                        columns = {'Counts of Country Availability'}) #Create a m_ctr_df
m_ctr_df.sort_values(by = 'Counts of Country Availability',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_m_ctr = m_ctr_df[0:10] 

> - Thirdly, **I draw a bar chart** to better understand which countries have a larger availability about the Netflix Movies.

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_ctr.index,
    y = top_10_m_ctr['Counts of Country Availability'],
    text = top_10_m_ctr['Counts of Country Availability'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which countries have a larger Movie Availability?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Countries',
                  titlefont_size=14))

-----------

# 2.7 Duration  ⏲⏳

> 📌 Note: Nowadays we are so busy during the day that you have to try to fit everything together if you want to do all the things to do. This also applies to extras such as seeing a movie or a TV series. However, they have a substantial difference that the film is one and therefore normally lasts an hour or more while the TV series that are episodic, which therefore they can be seen a little at a time or all in one go, they are easier as the average episodes last about 30/40 minutes. What do you prefer? Are you from #TeamMovie📽‍️ or #TeamSerie📺 ?

But let's find out what this dataframe teaches us about the duration of the various TV series or films

**In the Series 📺 world ...**

> Code:

> - First of all I create a new series dataframe **dropping the missing values contained in the "Runtime" column** and then I counts the values with the function [.value_counts()](https://medium.com/swlh/python-data-analysis-value-counts-115c7ad4ffec#:~:text=General%20usage.%20The%20value_counts%20%28%29%20function%20can%20be,or%20descending%20order%20%2C%20use%20sort%20as%20argument.).

In [ ]:
df_series_drt = df_series.dropna(subset=['Runtime'])
df_series_drt['Runtime'].value_counts()

**... we can see this dataframe telling us that Netflix series episodes are always 30 minutes or less.**

**Meanwhile in the Movie 📽‍️ universe ...**

> Code:

> - First of all I create a new series dataframe **dropping the missing values contained in the "Runtime" column**.

In [ ]:
df_movie_drt = df_movie.dropna(subset=['Runtime'])

> - Secondly, **I draw a bar chart** to better understand what is the average duration of the Movies.

In [ ]:
count_d = df_movie_drt['Runtime'].value_counts()
fig = go.Figure(data=[go.Bar(
    x = count_d.index,
    y = count_d,
    text = count_d,
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'What is the average duration of a movie on Netflix?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Duration',
                  titlefont_size=14))

... As we can see the great majority of the movies are **1-2 hour**

---------------------------

# 2.8 Directors 🎬 and actors 🤵 

> 📌 Note: Obviously, behind the creation of a film or a TV series there are many people who also work in the rearguards, but let's say that those who put their face on it and influence the success of the product in the foreground are the actors and the director. What is the role that fascinates you the most? Are you from #TeamDirector or #TeamActor?

So let's analyze and observe which are the most frequent directors and actors in the Netflix catalog

**2.8.1 Directors** 🎬

- **In Series 📺**

In [ ]:
df_series_dir = df_series.dropna(subset=['Director'])

In [ ]:
series_dir_list = df_series_dir['Director'].str.split(', ') #split the list into names
s_dir_list = {} #create an empty list
for directors in series_dir_list: # for any countries in series_ctr_list
    for director in directors: # for any country in countries
        if (director in s_dir_list): #if this country is already present in the s_ctr_list
            s_dir_list[director]+=1 # increase his value
        else:  # else
            s_dir_list[director]=1 # Create his index in the list
s_dir_df = pd.DataFrame(s_dir_list.values(),index = s_dir_list.keys(),
                        columns = {'Director Counts'}) #Create a s_ctr_df
s_dir_df.sort_values(by = 'Director Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_s_dir = s_dir_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_s_dir.index,
    y = top_10_s_dir['Director Counts'],
    text = top_10_s_dir['Director Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the directors that have directed more Tv Series?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of Tv Series he/she directed',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Directors',
                  titlefont_size=14))

As we can see the directors that have directed more Series Tv are **George Lucas**, **Hideki Takeuchi** and **Arcie Baron**.

- **In the Movies** 📽‍️

In [ ]:
df_movie_dir = df_movie.dropna(subset=['Director'])

In [ ]:
movie_dir_list = df_movie_dir['Director'].str.split(', ') #split the list into names
m_dir_list = {} #create an empty list
for directors in movie_dir_list: # for any countries in series_ctr_list
    for director in directors: # for any country in countries
        if (director in m_dir_list): #if this country is already present in the s_ctr_list
            m_dir_list[director]+=1 # increase his value
        else:  # else
            m_dir_list[director]=1 # Create his index in the list
m_dir_df = pd.DataFrame(m_dir_list.values(),index = m_dir_list.keys(),
                        columns = {'Director Counts'}) #Create a s_ctr_df
m_dir_df.sort_values(by = 'Director Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_m_dir = m_dir_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_dir.index,
    y = top_10_m_dir['Director Counts'],
    text = top_10_m_dir['Director Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the directors that have directed more Movies?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of Movies he/she directed',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Directors',
                  titlefont_size=14))

As we can see the directors that have directed more Movies are **Steven Spielberg**, **Johnnie To** and **Jing Wong**.

**2.8.2 Actors**

- **In Series planet**

In [ ]:
df_series_act = df_series.dropna(subset=['Actors'])

In [ ]:
series_act_list = df_series_act['Actors'].str.split(', ') #split the list into names
s_act_list = {} #create an empty list
for actors in series_act_list: # for any countries in series_ctr_list
    for actor in actors: # for any country in countries
        if (actor in s_act_list): #if this country is already present in the s_ctr_list
            s_act_list[actor]+=1 # increase his value
        else:  # else
            s_act_list[actor]=1 # Create his index in the list
s_act_df = pd.DataFrame(s_act_list.values(),index = s_act_list.keys(),
                        columns = {'Actor Counts'}) #Create a s_ctr_df
s_act_df.sort_values(by = 'Actor Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_s_act = s_act_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_s_act.index,
    y = top_10_s_act['Actor Counts'],
    text = top_10_s_act['Actor Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the actors that have played more Tv Series?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of Tv Series he/she played',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Actors',
                  titlefont_size=14))

As we can see the actors that have played more Series are **Johnny Yong Bosch**, **Kana Hanazawa** and **Ai Kayano**.

- **Meanwhile in the Movies world**

In [ ]:
df_movie_act = df_movie.dropna(subset=['Actors'])

In [ ]:
movie_act_list = df_movie_act['Actors'].str.split(', ') #split the list into names
m_act_list = {} #create an empty list
for actors in movie_act_list: # for any countries in series_ctr_list
    for actor in actors: # for any country in countries
        if (actor in m_act_list): #if this country is already present in the s_ctr_list
            m_act_list[actor]+=1 # increase his value
        else:  # else
            m_act_list[actor]=1 # Create his index in the list
m_act_df = pd.DataFrame(m_act_list.values(),index = m_act_list.keys(),
                        columns = {'Actor Counts'}) #Create a s_ctr_df
m_act_df.sort_values(by = 'Actor Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_m_act = m_act_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_act.index,
    y = top_10_m_act['Actor Counts'],
    text = top_10_m_act['Actor Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the actors that have played more Movies?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of Movies he/she played',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Actors',
                  titlefont_size=14))

As we can see the actors that have played more Movies are **Bruce Willis**, **Jackie Chan** and **Nicolas Cage**.

# 2.9 Tv Series 📺 and Movie 📽‍️ Charts by Rotten Tomatoes and IMBd

> 📌 Note: We are not human if we do not judge things, it is in our nature. As happens in all things, even in the film world there are those who give votes, for this reason we are going to analyze 2 of the major websites where you can find reliable reviews and votes that could help us, or advise us, in the search for a Film / TV series to see:
> * **Rotten Tomatoes** is an American review-aggregation website for film and television. The company was launched in August 1998 by three undergraduate students at the University of California, Berkeley: Senh Duong, Patrick Y. Lee, and Stephen Wang. Although the name "Rotten Tomatoes" connects to the practice of audiences throwing rotten tomatoes when disapproving of a poor stage performance, the original inspiration comes from a scene featuring tomatoes in the Canadian film Léolo (1992).
> * **IMDb (an acronym for Internet Movie Database)** is an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews.  Originally a fan-operated website, the database is now owned and operated by IMDb.com, Inc., a subsidiary of Amazon.

**2.9.1 Tv Series 📺 Charts**

- **IMDb**

In [ ]:
df_series_imdb = df_series.dropna(subset=['IMDb Score'])
df_series_imdb = df_series_imdb.sort_values(by = 'IMDb Score', ascending = False)
top_s_imdb_10_list = df_series_imdb[:10]

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_s_imdb_10_list['Title'],
    y = top_s_imdb_10_list['IMDb Score'],
    text = top_s_imdb_10_list['IMDb Score'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most rated Tv Series for IMDb?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='IMDb Score',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

- **Rotten Tomatoes**

In [ ]:
df_series_rt = df_series.dropna(subset=['Rotten Tomatoes Score'])
df_series_rt = df_series_rt.sort_values(by = 'Rotten Tomatoes Score', ascending = False)
top_s_rt_10_list = df_series_rt[:10]

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_s_rt_10_list['Title'],
    y = top_s_rt_10_list['Rotten Tomatoes Score'],
    text = top_s_rt_10_list['Rotten Tomatoes Score'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most rated Tv Series for Rotten Tomatoes?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Rotten Tomatoes Score',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

As we can see, **the rankings of the top 10 for the series are completely different**, so before choosing a movie to watch it is best to take a look at both sites so that you can have a comparison and have a wider view

**2.9.2 Movie 📽‍️ Charts**

- **IMDb**

In [ ]:
df_movie_imdb = df_movie.dropna(subset=['IMDb Score'])
df_movie_imdb = df_movie_imdb.sort_values(by = 'IMDb Score', ascending = False)
top_m_imdb_10_list = df_movie_imdb[:10]

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_m_imdb_10_list['Title'],
    y = top_m_imdb_10_list['IMDb Score'],
    text = top_m_imdb_10_list['IMDb Score'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most rated Movies for IMDb?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='IMDb Score',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

- **Rotten Tomatoes**

In [ ]:
df_movie_rt = df_movie.dropna(subset=['Rotten Tomatoes Score'])
df_movie_rt = df_movie_rt.sort_values(by = 'Rotten Tomatoes Score', ascending = False)
top_m_rt_10_list = df_movie_rt[:10]

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_m_rt_10_list['Title'],
    y = top_m_rt_10_list['Rotten Tomatoes Score'],
    text = top_m_rt_10_list['Rotten Tomatoes Score'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most rated Movies for Rotten Tomatoes?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Rotten Tomatoes Score',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

As we had seen for the Series, even in the world of films **the rankings of the top 10 are very different** and therefore it is better to look at both sites to have a comparison and a better view on the product

# 2.10 Top 15 Tv Series and Movie for awards received  🏆 🥇

> Obviously, in addition to the votes, there are also prizes up for grabs in the film world and among the most famous are the Golden Globes, the Hollywood Film Awards and the Oscars.

So let's go see which are the most awarded films and series

- **In the Tv Series  📺 Universe**

In [ ]:
df_series_awr = df_series.dropna(subset=['Awards Received'])
df_series_awr = df_series_awr.sort_values(by = 'Awards Received', ascending = False)
top_s_awr_15_list = df_series_awr[:15]

In [ ]:
colors_15 = ['DarkRed', 'FireBrick', 'FireBrick' ,'Red', 'Crimson', 'Crimson', 'IndianRed' , 'slategray', 'slategray' , 'gray' , 'gray', 'dimgrey', 'dimgrey', 'DarkSlateGrey', 'black']
fig = go.Figure(data=[go.Bar(
    x = top_s_awr_15_list['Title'],
    y = top_s_awr_15_list['Awards Received'],
    text = top_s_awr_15_list['Awards Received'],
    textposition='auto',
    marker_color=colors_15 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the 15 TV series 📺 that have received the most awards?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Awards Received',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

- **While in the movie 📽‍️ catalog**

In [ ]:
df_movie_awr = df_movie.dropna(subset=['Awards Received'])
df_movie_awr = df_movie_awr.sort_values(by = 'Awards Received', ascending = False)
top_m_awr_15_list = df_movie_awr[:15]

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_m_awr_15_list['Title'],
    y = top_m_awr_15_list['Awards Received'],
    text = top_m_awr_15_list['Awards Received'],
    textposition='auto',
    marker_color=colors_15 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the 15 Movies 📽‍️ that have received the most awards?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='Awards Received',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

# 2.11 Productivity per year (on Netflix) 📆 

> And finally, as a last topic, let's analyze which was the year in which the most series were released and which was the most films released. What do you think they were?

- **2.11.1 Tv Series 📺**

In [ ]:
df_series_rel = df_series.dropna(subset=['Released_Year_Net'])

In [ ]:
series_rel_list = df_series_act['Released_Year_Net']
s_rel_list = {} #create an empty list
for year in series_rel_list: # for any year in series_rel_list
    if (year in s_rel_list): #if this year is already present in the s_rel_list
        s_rel_list[year]+=1 # increase his value
    else:  # else
        s_rel_list[year]=1 # Create his index in the list
s_rel_df = pd.DataFrame(s_rel_list.values(),index = s_rel_list.keys(),
                        columns = {'Year Counts'}) #Create a s_ctr_df
s_rel_df.sort_values(by = 'Year Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_s_rel = s_rel_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_s_rel.index,
    y = top_10_s_rel['Year Counts'],
    text = top_10_s_rel['Year Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'In which year there have been more Tv Series released ?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of release',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

- Movie 📽‍️

In [ ]:
df_movie_rel = df_movie.dropna(subset=['Released_Year_Net'])

In [ ]:
movie_rel_list = df_movie_act['Released_Year_Net']
m_rel_list = {} #create an empty list
for year in movie_rel_list: # for any year in movie_rel_list
    if (year in m_rel_list): #if this year is already present in the m_rel_list
        m_rel_list[year]+=1 # increase his value
    else:  # else
        m_rel_list[year]=1 # Create his index in the list
m_rel_df = pd.DataFrame(m_rel_list.values(),index = m_rel_list.keys(),
                        columns = {'Year Counts'}) #Create a s_ctr_df
m_rel_df.sort_values(by = 'Year Counts',ascending = False,inplace = True) #Sort the dataframe in ascending order
top_10_m_rel = m_rel_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_rel.index,
    y = top_10_m_rel['Year Counts'],
    text = top_10_m_rel['Year Counts'],
    textposition='auto',
    marker_color=colors_10 # marker color can be a single color value or an iterable
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'In which year there have been more Movies released ?',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  yaxis=dict(
                  title='N° of release',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Titles',
                  titlefont_size=14))

**Thank you so much for looking at this notebook, I hope you enjoyed it and if so I would invite you to put an upvote. If you have found any errors, please write them to me in the comments or even if you have any suggestions for improving the notebook. thank you very much again and good Kaggling!**